###PRELIMINARIES STEPS

In [ ]:
pip install z3-solver

     |████████████████████████████████| 33.0MB 84kB/s 


In [ ]:
pip install utils

In [ ]:
from z3 import *
from utils import *
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from random import random
%matplotlib inline

In [ ]:
txtfiles = []
for file in glob.glob("*.txt"):
    txtfiles.append(file)
print(txtfiles)

['ins-11.txt', 'ins-13.txt']


In [ ]:
w=0
n_circuits=0
circuits=[]

with open(str(txtfiles[0]),'r') as f:
    lines= f.readlines()
    w=int(lines[0])
    n_circuits=int(lines[1])
    lines_2=[]
    for x in lines[2:]:
      newel=x.strip("\n").split(" ")
      lines_2.append(newel)
    for i in range(len(lines_2)):
      circuits.append([int(lines_2[i][0]),int(lines_2[i][1])])

print("Maximum length: " +str(w) + "\nNumber of circuits: " +str(n_circuits)+ "\nDimension of circuits: "+str(circuits))

Maximum length: 18
Number of circuits: 16
Dimension of circuits: [[3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 10], [3, 11], [4, 3], [4, 4], [4, 5], [4, 6], [5, 3], [5, 4], [5, 5], [5, 6]]


### FUNCTIONS

In [ ]:
def display_solution(solution,w,n):
  ## Output
  print(f'''{n}
{w} {solution[0]['length = ']}''')
  for j in range(1,n+1):
    print(f'''{solution[j]['x_dimension = ']} {solution[j]['y_dimension = ']} {solution[j]['x_position = ']} {solution[j]['y_position = ']}, {solution[j]['rotation = ']}''')

  boxes = []
  for i in range(1,n+1):
    circ = []
    for el in solution[i].keys():
      circ.append(solution[i][el])
    boxes.append(circ)

  fig1 = plt.figure()
  ax1 = fig1.add_subplot(111, aspect='equal')

  for i,box in enumerate(boxes):
    position = (box[2],box[3])
    heigh = box[1]
    width = box[0]
    ax1.add_patch(
      patches.Rectangle(position, width, heigh, 
                        color = (random(),random(),random())))
  
  plt.ylim((0,solution[0]['length = ']));
  plt.xlim((0,w));


In [ ]:
def lex_less(vars_1,vars_2):
  n = len(vars_1)
  sol = [vars_1[0] <= vars_2[0]]

  for j in range(1,n): #equations
    el = 1
    eq = [vars_1[0] == vars_2[0]]
    while el < j: # var in equations
      temp = vars_1[el] == vars_2[el]
      el += 1
      eq.append(temp)

    eq = Implies(And(eq),vars_1[j]<=vars_2[j])
    sol.append(eq)

  return And(sol)

def lim_length(vars):
    max = vars[0]
    for el in vars[1:]:
        max = If(el > max, el, max)
    return max
    


### PROBLEM SPECIFICATION

In [ ]:
def problem(lim,istance):
    #variables
    pos=[]
    dim = []
    rotation = []
    for i, ist in enumerate(istance):
        pos.append([])
        dim.append([])
        rotation.append(Int(f"rot{i}"))
        for j, _ in enumerate(ist):
            pos[i].append(Int(f"p{i}_{j}")) #position variables
            dim[i].append(Int(f"d{i}_{j}")) #dimensinb variables

    # Solver
    opt = Optimize()
    opt.set("timeout", 300000) # time limit = 5 min

    # rotation constraint
    for i,ist in enumerate(istance):
      opt.add(Or(rotation[i] == 0, rotation[i] == 1))
      opt.add(Implies(ist[1] > lim,rotation[i] == 0))

    # dimension constraint
    for i in range(len(istance)):
      opt.add(If(rotation[i] == 0, 
                 And(dim[i][0] == istance[i][0],dim[i][1] == istance[i][1]),
                 And(dim[i][0] == istance[i][1], dim[i][1] == istance[i][0])))
    
    #Position >=0
    for ist in pos:
        for position in ist:
            opt.add(position >= 0)

    #X_position <= w
    for i, ist in enumerate(pos):
        opt.add(ist[0]+dim[i][0]<=lim)
    
    #No overlap
    for i in range(len(pos)):
        for j in range(len(pos)):
            if i!=j:
                opt.add(Or(Or(pos[i][0]+dim[i][0]<=pos[j][0],pos[j][0]
                +dim[j][0]<=pos[i][0]),
                Or(pos[i][1]+dim[i][1]<=pos[j][1],pos[j][1]+dim[j][1]
                <=pos[i][1])))

    # lexicographic constr symmetry breaking
    x_pos = [p[0] for p in pos ]
    y_pos = [p[1] for p in pos ]
    x_pos_symmetry = [lim - el[0] - p[0] for el, p in zip(dim, pos)]
    y_pos_symmetry = [lim - el[1] - p[1] for el, p in zip(dim, pos)]

    opt.add(lex_less(x_pos,x_pos_symmetry))
    opt.add(lex_less(y_pos,y_pos_symmetry))
    

    #Objective
    limit = [d[-1] + ist[-1] for ist, d in zip(dim, pos)]
    makespan = Int('makespan')
    opt.add(makespan <= sum([max(ist) for ist in istance]))
    objective = makespan == lim_length(limit)
    opt.add(objective)
    opt.minimize(makespan)

    opt.check()
    m = opt.model()
    sol=[]
    length = int(m.evaluate(makespan).as_string())
    sol.append({"length = " : length})
    for i in range(len(istance)):
      x_position= int(m.evaluate(pos[i][0]).as_string())
      y_position= int(m.evaluate(pos[i][1]).as_string())
      x_dimension = int(m.evaluate(dim[i][0]).as_string())
      y_dimension = int(m.evaluate(dim[i][1]).as_string())
      r = m.evaluate(rotation[i])

      sol.append({
            "x_dimension = " : x_dimension,
            "y_dimension = " : y_dimension,
            "x_position = " : x_position,
            "y_position = " : y_position,
            "rotation = ": r
        })

    return sol

### INSTANCE RESOLUTION


In [ ]:
display_solution(problem(w,circuits),w,n_circuits)

In [ ]:
# for i in range(2):
#   w=0
#   n_circuits=0
#   circuits=[]

#   with open(str(txtfiles[i]),'r') as f:
#       lines= f.readlines()
#       w=int(lines[0])
#       n_circuits=int(lines[1])
#       lines_2=[]
#       for x in lines[2:]:
#         newel=x.strip("\n").split(" ")
#         lines_2.append(newel)
#       for i in range(len(lines_2)):
#         circuits.append([int(lines_2[i][0]),int(lines_2[i][1])])
#   #print("Doing: " +str(txtfiles[i])+ "\n")

#   print("Maximum length: " +str(w) + "\nNumber of circuits: " +str(n_circuits)+ "\nDimension of circuits: "+str(circuits)+ "\n")
#   display_solution(problem(w,circuits),w,n_circuits)
#   print("-------------------------------------------------------------------------------------------------------\n")